In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


# NORMAL MODEL

In [3]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_training_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
train = get_training_data('G:/#Kuliah/Skripsi/Skripsi/Code/Coding_Skripsi/data/train')      
test = get_training_data('G:/#Kuliah/Skripsi/Skripsi/Code/Coding_Skripsi/data/test')
val = get_training_data('G:/#Kuliah/Skripsi/Skripsi/Code/Coding_Skripsi/data/val')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15224\771166761.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [5]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
    
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

In [6]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [7]:
# resize data for deep learning 
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [8]:
def load_augmented_data(augmented_dir, label, img_size):
    augmented_data = []
    for img in os.listdir(augmented_dir):
        try:
            img_arr = cv2.imread(os.path.join(augmented_dir, img), cv2.IMREAD_GRAYSCALE)
            resized_arr = cv2.resize(img_arr, (img_size, img_size))
            augmented_data.append([resized_arr, label])
        except Exception as e:
            print(e)
    return np.array(augmented_data)

# Define parameters
data_dir = '../data/train'  # Adjust this path to your dataset
save_dir = 'G:/#Kuliah/Skripsi/Skripsi/Code/Coding_Skripsi/data/augmented_normal/NORMAL'  # Directory to save augmented images
label = 'NORMAL'  # Class label to augment
img_size = 150  # Image size (adjust if different)
additional_images_needed = 2690  # Number of additional images needed
batch_size = 32  # Number of images to generate per original image

augmented_normal_data = load_augmented_data(save_dir, label=1, img_size=img_size)

# Convert to appropriate format and normalize
x_augmented_normal = np.array([i[0] for i in augmented_normal_data]) / 255
x_augmented_normal = x_augmented_normal.reshape(-1, img_size, img_size, 1)
y_augmented_normal = np.array([i[1] for i in augmented_normal_data])

# Append to existing training data
x_train = np.concatenate((x_train, x_augmented_normal), axis=0)
y_train = np.concatenate((y_train, y_augmented_normal), axis=0)

# Shuffle the dataset
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

# Count after augmentation
print("Count after augmentation:", len(x_train))



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15224\2657222256.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(augmented_data)


Count after augmentation: 7069


In [9]:
# bestParam = [15,48,88,3,5,5]
bestParam = [33,47,73,4,8,20]

In [10]:
model = Sequential()
model.add(Conv2D(bestParam[0] , (bestParam[3],bestParam[3]) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(bestParam[1] , (bestParam[4],bestParam[4]) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(bestParam[2] , (bestParam[5],bestParam[5]) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = "adam" , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 33)      561       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 33)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 47)        99311     
                                                                 
 dropout (Dropout)           (None, 75, 75, 47)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 47)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 38, 38, 73)        1

In [11]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

In [12]:
# # Adjust the range to match the number of epochs in your history data
# epochs = [i for i in range(len(history.history['accuracy']))]

# fig, ax = plt.subplots(1, 2, figsize=(20, 10))

# train_acc = history.history['accuracy']
# train_loss = history.history['loss']
# val_acc = history.history['val_accuracy']
# val_loss = history.history['val_loss']

# ax[0].plot(epochs, train_acc, 'go-', label='Training Accuracy')
# ax[0].plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
# ax[0].set_title('Training & Validation Accuracy')
# ax[0].legend()
# ax[0].set_xlabel("Epochs")
# ax[0].set_ylabel("Accuracy")

# ax[1].plot(epochs, train_loss, 'g-o', label='Training Loss')
# ax[1].plot(epochs, val_loss, 'r-o', label='Validation Loss')
# ax[1].set_title('Training & Validation Loss')
# ax[1].legend()
# ax[1].set_xlabel("Epochs")
# ax[1].set_ylabel("Loss")

# plt.show()


In [13]:
from sklearn.metrics import classification_report
history = model.fit(x_train,y_train, batch_size = 64 ,epochs = 10 , validation_data = [x_val, y_val] ,callbacks = [learning_rate_reduction])

predictions = model.predict(x_test)
predictions = predictions.reshape(1,-1)[0]
predictions[:15]



binary_predictions = [1 if i > 0.5 else 0 for i in predictions]
cm = confusion_matrix(y_test,binary_predictions)

True_Positive = []
True_Negative = []
False_Positive = []
False_negative = []
acc = []
error = []
precision = []
recall = []
TP_rate = []
FP_rate = []
specificity = []
wrong_prediction = []
F1 = []

TP = cm[0][0]  # True Positives
TN = cm[1][1]  # True Negatives
FP = cm[0][1]  # False Positives
FN = cm[1][0]  # False Negatives
classification_accuracy = (TP + TN) / (TP + TN + FP + FN)
classification_error = (FP + FN) / (TP + TN + FP + FN)
precision_calc = TP / (TP + FP)
recall_calc = TP / (TP + FN)
true_positive_rate = TP / (TP + FN)
false_positive_rate = FP / (FP + TN)
specificity_calc = TN / (TN + FP)
F1_score = 2 * (precision_calc * recall_calc) / (precision_calc + recall_calc)
wrong_calc = FN + FP

True_Positive.append(TP)
True_Negative.append(TN)
False_Positive.append(FP)
False_negative.append(FN)
acc.append(classification_accuracy)
error.append(classification_error)
precision.append(precision_calc)
recall.append(recall_calc)
TP_rate.append(true_positive_rate)
FP_rate.append(false_positive_rate)
specificity.append(specificity_calc)
F1.append(F1_score)
wrong_prediction.append(wrong_calc)

df = {
    'TP': True_Positive,
    'TN': True_Negative,
    'FP': False_Positive,
    'FN': False_negative,
    'acc': acc,
    'error': error,
    'precision': precision,
    'recall': recall,
    'TP_rate': TP_rate,
    'FP_rate': FP_rate,
    'specificity': specificity,
    'F1_score': F1,
    'wrong_prediction' : wrong_prediction
}

normal_trial = pd.read_csv('normal_trial2.csv')

df = pd.DataFrame(df)

normal_trial = pd.read_csv('normal_trial2.csv')

# Concatenate df with normal_trial along the row axis
combined_df = pd.concat([normal_trial, df], ignore_index=True)

# Save the combined DataFrame back to the CSV file
combined_df.to_csv('normal_trial2.csv', index=False)

    


Epoch 1/10
111/111 [==============================] - 24s 137ms/step - loss: 0.5143 - accuracy: 0.7294 - val_loss: 0.1695 - val_accuracy: 0.9385 - lr: 0.0010
Epoch 2/10
111/111 [==============================] - 12s 109ms/step - loss: 0.1501 - accuracy: 0.9444 - val_loss: 0.1467 - val_accuracy: 0.9487 - lr: 0.0010
Epoch 3/10
111/111 [==============================] - 12s 110ms/step - loss: 0.1207 - accuracy: 0.9571 - val_loss: 0.0983 - val_accuracy: 0.9761 - lr: 0.0010
Epoch 4/10
111/111 [==============================] - 12s 110ms/step - loss: 0.0843 - accuracy: 0.9709 - val_loss: 0.0911 - val_accuracy: 0.9675 - lr: 0.0010
Epoch 5/10
111/111 [==============================] - ETA: 0s - loss: 0.0729 - accuracy: 0.9726
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
111/111 [==============================] - 12s 110ms/step - loss: 0.0729 - accuracy: 0.9726 - val_loss: 0.0834 - val_accuracy: 0.9761 - lr: 0.0010
Epoch 6/10
111/111 [=============================